# Libraries

In [58]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
import pandas as pd
import psycopg2
import os

In [59]:
load_dotenv()

True

# Database

In [60]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DB = os.getenv('POSTGRES_DB')

In [61]:
engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

In [62]:
Session = sessionmaker(bind=engine)
session = Session()

# Data

In [66]:
sql_query = text("""
WITH global_climate_and_forest_fire AS (
    SELECT * FROM global_climate 
    JOIN forest_fire USING(latitude, longitude, \"date\")
), 
global_climate_and_forest_fire_and_ndvi AS (
    SELECT * FROM global_climate_and_forest_fire 
    JOIN ndvi USING(latitude, longitude, \"date\")
), 
population_density_and_land_cover AS (
    SELECT * FROM population_density 
    JOIN land_cover USING(latitude, longitude, \"year\")
), 
all_data AS (
    SELECT gc_ff_nd.*, pd_lc.land_cover, pd_lc.population_density
    FROM population_density_and_land_cover pd_lc
    JOIN global_climate_and_forest_fire_and_ndvi gc_ff_nd
    USING(latitude, longitude) 
    WHERE pd_lc.\"year\" = EXTRACT(YEAR FROM gc_ff_nd.\"date\")
)

SELECT * FROM all_data;
""")
data = session.execute(sql_query)

In [67]:
rows = data.fetchall()
df = pd.DataFrame(rows, columns=data.keys())

In [77]:
df.head(5)

,latitude,longitude,date,ws,vpd,vap,tmin,tmax,swe,srad,...,bright_t31,frp,daynight,type,n_pixels,vim,vim_avg,viq,land_cover,population_density
0,8.4004,-73.6058,2002-07-03,2.02,1.01,2.822,21.92,32.88,0.0,209.1,...,297.9,12.7,D,0,29.0,0.7081,0.7544,94.2474,28,29.434807
1,5.9399,-69.5183,2002-08-17,1.16,0.86,2.988,22.98,32.37,0.0,194.0,...,296.9,19.5,D,0,597.0,0.6282,0.6340,99.1554,23,0.441092
2,9.5491,-75.3100,2002-08-17,2.19,0.67,2.791,22.47,29.67,0.0,214.3,...,297.5,107.0,D,0,2.0,0.7964,0.8101,98.4076,24,177.031860
3,5.6388,-68.3433,2002-08-17,1.08,0.98,3.016,23.47,33.10,0.0,193.5,...,301.6,33.0,D,0,392.0,0.5832,0.5938,98.3554,24,0.216183
4,5.8698,-75.0852,2002-08-17,0.96,0.75,2.225,18.34,28.13,0.0,200.6,...,294.5,50.2,D,0,7.0,0.8105,0.8035,100.8241,42,4.816177


# Model

# Close connection

In [57]:
session.close()
engine.dispose()